# RSA benchmark

Create a Python program that uses the RSA implementation you created in exercise 5 and the AES implementation you created on assignment 2. For RSA you should use OAEP padding, and a 1024-bit key, a 2048-bit key, and a 4096-bit key. For AES you should use CTR mode with a 256 bits key.


For these benchmarks, you should try calculating the time it takes to encrypt about 100MB of randomly generated byte data using each of the configurations listed above.

In [ ]:
import timeit
import argparse
import sys
from cryptography.hazmat.primitives.asymmetric.rsa import generate_private_key
import os
import secrets
from statistics import stdev, mean
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes


from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.serialization import load_pem_private_key, load_pem_public_key
from cryptography.hazmat.primitives.asymmetric.rsa import generate_private_key, RSAPublicKey, RSAPrivateKey
from cryptography.hazmat.primitives.asymmetric import padding



In [ ]:
def encrypt_OAEP(plain: bytes, public_key: RSAPublicKey, block_size: int | None = None):
    if block_size is None:
        block_size = (public_key.public_numbers(
        ).n.bit_length()+7)//8-2*256//8-2

    blocks = [plain[i:i+block_size] for i in range(0, len(plain), block_size)]

    return b''.join([public_key.encrypt(block, padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None)) for block in blocks])

def encrypt_CTR(plain: bytes, key: bytes, nonce: bytes):
    cipher = Cipher(algorithms.AES256(key), modes.CTR(nonce))
    encryptor = cipher.encryptor()
    return encryptor.update(plain) + encryptor.finalize()



In [ ]:
def benchmark_RSA(key_size: int):
    times = []
    for _ in range(10):
        data = os.urandom(100*1024*1024)
        key = generate_private_key(0x10001, key_size).public_key()
        times.append(timeit.timeit(lambda: encrypt_OAEP(data, key), number=1))
    return mean(times), stdev(times)


def benchmark_AES():
    times = []
    for _ in range(10):
        data = os.urandom(100*1024*1024)
        key = secrets.token_bytes(32)
        nonce = secrets.token_bytes(16)

        times.append(timeit.timeit(
            lambda: encrypt_CTR(data, key, nonce), number=1))
    return mean(times), stdev(times)


def main():
    aes_mean, aes_stdev = benchmark_AES()
    print(f"AES: Mean {aes_mean}, Standard deviation {aes_stdev}")

    for key_size in {1024, 2048, 4096}:
        rsa_mean, rsa_stdev = benchmark_RSA(key_size)
        print(
            f"RSA {key_size}-bit key: Mean {rsa_mean}, Standard deviation {rsa_stdev}")



In [ ]:

if __name__ == "__main__":
    main()